In [1]:
import sys
if "../../" not in sys.path:
    sys.path.append("../../")

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import string
from importlib import reload
from collections import Counter
from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm as tqdm_apply
tqdm_apply.pandas()

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time

import pdaactconn as pc
from trialexplorer import AACTStudySet

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import nltk
from nltk.corpus import stopwords
stop_words_nltk = set(stopwords.words('english'))

In [4]:
# loading all interventional studies
conn = pc.AACTConnection(source=pc.AACTConnection.REMOTE)
ss = AACTStudySet.AACTStudySet(conn=conn, 
                               tqdm_handler=tqdm)
ss.add_constraint("study_type = 'Interventional'")
ss.load_studies()

262271 studies loaded!


In [5]:
# adding and loading dimensional data
ss.add_dimensions(['sponsors',
                   'result_groups'])
ss.refresh_dim_data()

Successfuly added these 2 dimensions: ['sponsors', 'result_groups']
Failed to add these 0 dimensions: []


Syncing the temp table temp_cur_studies in 525 chunks x 500 records each

Creating index on the temp table
 - Loading dimension sponsors
 -- Loading raw data
 -- Sorting index
 - Loading dimension result_groups
 -- Loading raw data
 -- Sorting index


In [6]:
rg = ss.dimensions['result_groups'].data
sp = ss.dimensions['sponsors'].data

### Filtering for those studies with results

In [7]:
res_idx = rg.index.get_level_values(0).unique()

In [8]:
ss.studies = ss.studies.loc[res_idx]

In [9]:
# refreshing so that sponsors only include result-containing studies
ss.refresh_dim_data()

Syncing the temp table temp_cur_studies in 79 chunks x 500 records each

Creating index on the temp table
 - Loading dimension sponsors
 -- Loading raw data
 -- Sorting index
 - Loading dimension result_groups
 -- Loading raw data
 -- Sorting index


In [10]:
sp = ss.dimensions['sponsors'].data

In [11]:
all_ind = sp[sp['agency_class'] == 'Industry'].copy()
uniq_names = list(all_ind['name'].unique())
len(uniq_names)

2477

In [12]:
all_ind['name'] = all_ind['name'].apply(lambda x: x.lower())

In [13]:
pharma_counts = all_ind.groupby('name').size().sort_values(ascending=False)

In [14]:
df_ct = pd.DataFrame(pharma_counts, columns=['count']).reset_index()
df_ct

,name,count
0,glaxosmithkline,1704
1,merck sharp & dohme corp.,1357
2,pfizer,1239
3,eli lilly and company,1026
4,novartis pharmaceuticals,884
...,...,...
2472,provectus pharmaceuticals,1
2473,"entrinsic health solutions, inc.",1
2474,envisia therapeutics,1
2475,envoy medical corporation,1


Looking at the companies that researched the top 50% of studies, we can see that they are located in:
- mostly US
- Some Swiss (Roche, Novartis)
- Some German (Bayer)
- Some French (Sanofi)

We'll start with the US ones for now:

## From ETF holdings

If we have more stocks to add, add them here

In [99]:
df_iwm = pd.read_csv('IWM_holdings.csv', skiprows=9)
df_iwb = pd.read_csv('IWB_holdings.csv', skiprows=9)
df_exsa = pd.read_csv('EXSA_holdings.csv', skiprows=2)
df_ewj = pd.read_csv('EWJ_holdings.csv', skiprows=9)
df_ewa = pd.read_csv('EWA_holdings.csv', skiprows=9)


dfj_iwm = df_iwm[df_iwm['Asset Class'] == 'Equity'][['Name', 'Ticker']]
dfj_iwm['source'] = 'Russell'

dfj_iwb = df_iwb[df_iwb['Asset Class'] == 'Equity'][['Name', 'Ticker']]
dfj_iwb['source'] = 'Russell'

df_exsa = df_exsa[df_exsa['Asset Class'] == 'Equity'][['Name', 
                                                       'Issuer Ticker']].rename(columns={'Issuer Ticker': 'Ticker'})
df_exsa['source'] = 'Stoxx'

df_ewj = df_ewj[df_ewj['Asset Class'] == 'Equity'][['Name', 'Ticker']]
df_ewj['source'] = 'Japan'

df_ewa = df_ewa[df_ewa['Asset Class'] == 'Equity'][['Name', 'Ticker']]
df_ewa['source'] = 'Australia'


In [100]:
df_stk = pd.concat([dfj_iwm, dfj_iwb, df_exsa, df_ewj, df_ewa], sort=False).reset_index(drop=True)
df_stk['Name'] = df_stk['Name'].apply(lambda x: x.lower())

In [108]:
df_stk.tail()

,Name,Ticker,source
3964,cimic group ltd,CIM,Australia
3965,wisetechglobal pty ltd,WTC,Australia
3966,harvey norman holdings ltd,HVN,Australia
3967,flight centre travel group ltd,FLT,Australia
3968,bgp holdings plc - unlisted prvt,3357358Z,Australia


In [109]:
df_stk.shape

(3969, 3)

## what are the stop words in this corpus?

In [110]:
def tokenize_and_count(list_names):
    c = Counter()
    for cur_name in list_names:
        tokens = cur_name.split()
        for t in tokens:
            c[t] += 1
    return c

In [111]:
c_stk = tokenize_and_count(df_stk['Name'])
c_ct = tokenize_and_count(df_ct['name'])

In [112]:
# looking through, these all look like valid stop words
stop_words_stocks = [x[0] for x in c_stk.most_common(30)]
stop_words_stocks

['inc',
 'corp',
 'class',
 'ltd',
 'group',
 'a',
 'holdings',
 'plc',
 'reit',
 'sa',
 'financial',
 'ag',
 'bancorp',
 'international',
 'therapeutics',
 'energy',
 'pharmaceuticals',
 'technologies',
 'b',
 'trust',
 'industries',
 'and',
 'bank',
 'holding',
 'nv',
 'services',
 'realty',
 'systems',
 'first',
 'national']

In [113]:
# looking through, 'merck' should not be a stop word
stop_words_ct = [x[0] for x in c_ct.most_common(30)]
stop_words_ct = [x for x in stop_words_ct if x != 'merck' and x != 'johnson']
stop_words_ct

['inc.',
 'pharmaceuticals',
 'ltd.',
 'llc',
 'medical',
 'corporation',
 'pharmaceuticals,',
 'pharma',
 'inc',
 'therapeutics',
 'ltd',
 'co.,',
 'therapeutics,',
 'pharmaceutical',
 'research',
 'gmbh',
 'limited',
 '&',
 'medical,',
 'company',
 'international',
 'technologies',
 'ag',
 'of',
 'health',
 'group',
 'a',
 's.a.']

### let's join all of the stopwords

In [114]:
all_stopwords = list(stop_words_nltk)
for w in stop_words_stocks:
    if w not in all_stopwords:
        all_stopwords.append(w)
        
for w in stop_words_ct:
    if w not in all_stopwords:
        all_stopwords.append(w)

len(all_stopwords)

225

## Removing stopwords from each corpus

In [115]:
def remove_stopwords(list_names, stop_words):
    rt_list = []
    orig_map = {}
        
    no_punct = str.maketrans('', '', string.punctuation)
    
    for cur_name in list_names:
        cur_name_no_punct = cur_name.translate(no_punct)
        cur_tokens = cur_name_no_punct.split()
        nostop_tokens = [x for x in cur_tokens if x not in stop_words]
        if len(nostop_tokens) > 0:
            no_stop = " ".join(nostop_tokens)
            rt_list.append(no_stop)
            orig_map[no_stop] = cur_name
        else:
            rt_list.append("")
    return rt_list

In [116]:
df_ct['to_match'] = remove_stopwords(df_ct['name'], all_stopwords)
df_ct.head(10)

,name,count,to_match,matched
0,glaxosmithkline,1704,glaxosmithkline,glaxosmithkline
1,merck sharp & dohme corp.,1357,merck sharp dohme,None
2,pfizer,1239,pfizer,pfizer
3,eli lilly and company,1026,eli lilly,eli lilly
4,novartis pharmaceuticals,884,novartis,novartis
5,astrazeneca,794,astrazeneca,astrazeneca
6,hoffmann-la roche,621,hoffmannla roche,None
7,boehringer ingelheim,584,boehringer ingelheim,None
8,"genentech, inc.",501,genentech,None
9,bristol-myers squibb,475,bristolmyers squibb,bristol myers squibb


In [117]:
df_stk['to_match'] = remove_stopwords(df_stk['Name'], all_stopwords)
df_stk.head(10)

,Name,Ticker,source,to_match
0,teladoc health inc,TDOC,Russell,teladoc
1,generac holdings inc,GNRC,Russell,generac
2,lumentum holdings inc,LITE,Russell,lumentum
3,trex inc,TREX,Russell,trex
4,novocure ltd,NVCR,Russell,novocure
5,rexford industrial realty reit inc,REXR,Russell,rexford industrial
6,haemonetics corp,HAE,Russell,haemonetics
7,portland general electric,POR,Russell,portland general electric
8,amedisys inc,AMED,Russell,amedisys
9,enphase energy inc,ENPH,Russell,enphase


## Matching the two lists

In [118]:
def gen_extract_fn(threshold, match_to_list):
    def extract_fn(in_str):
        res = process.extract(in_str, match_to_list, limit=1, scorer=fuzz.ratio)
        company_name = res[0][0]
        match_score = res[0][1]
        
        if match_score >= threshold:
            return company_name
        else:
            return None
    return extract_fn

In [119]:
df_ct['matched'] = df_ct['to_match'].apply(gen_extract_fn(90, df_stk['to_match']))

In [159]:
df_result = df_ct.drop('to_match', axis=1).merge(df_stk[['to_match', 'Ticker', 'source']], 
                                                 how='left',
                                                 left_on='matched',
                                                 right_on='to_match').drop('to_match', axis=1)

In [160]:
pct_match = df_result.dropna()['count'].sum() / df_result['count'].sum()
pct_match

0.45618044887406795

In [161]:
df_result.head()

,name,count,matched,Ticker,source
0,glaxosmithkline,1704,glaxosmithkline,GSK,Stoxx
1,merck sharp & dohme corp.,1357,None,NaN,NaN
2,pfizer,1239,pfizer,PFE,Russell
3,eli lilly and company,1026,eli lilly,LLY,Russell
4,novartis pharmaceuticals,884,novartis,NOVN,Stoxx


## What is still unmatched?

In [33]:
df_result[pd.isnull(df_result['matched'])].to_csv('unmatched.csv')

### Let's manually match these remaining

In [124]:
df_stk.to_csv('all_stocks.csv')

## manually mapped:

In [162]:
df_manual = pd.read_csv('manual_map.csv').dropna()
df_manual.head()

,name,count,manual match
0,merck sharp & dohme corp.,1357,merck co
1,hoffmann-la roche,621,roche par
3,"genentech, inc.",501,roche par
4,shire,232,takeda
5,forest laboratories,190,allergan


In [163]:
dict_man = df_manual.set_index('name')['manual match'].to_dict()

In [164]:
def combine_columns(c1, c2):
    final_c = []
    for i in range(len(c1)):
        if pd.isnull(c1[i]):
            final_c.append(c2[i])
        else:
            final_c.append(c1[i])
    return final_c

In [165]:
df_result['manual_map'] = df_result['name'].apply(lambda x: dict_man[x] if x in dict_man.keys() else None)

df_stk_man = df_stk[['to_match', 'Ticker', 'source']].add_prefix('man_')

df_result = df_result.merge(df_stk_man, how='left',
                            left_on='manual_map',
                            right_on='man_to_match')

df_result['matched_f'] = combine_columns(df_result['matched'].values, df_result['manual_map'].values)
df_result['Ticker_f'] = combine_columns(df_result['Ticker'].values, df_result['man_Ticker'].values)
df_result['source_f'] = combine_columns(df_result['source'].values, df_result['man_source'].values)
df_result = df_result[['name', 'count', 'matched_f', 'Ticker_f', 'source_f']]

In [166]:
df_result

,name,count,matched_f,Ticker_f,source_f
0,glaxosmithkline,1704,glaxosmithkline,GSK,Stoxx
1,merck sharp & dohme corp.,1357,merck co,MRK,Russell
2,pfizer,1239,pfizer,PFE,Russell
3,eli lilly and company,1026,eli lilly,LLY,Russell
4,novartis pharmaceuticals,884,novartis,NOVN,Stoxx
...,...,...,...,...,...
2503,provectus pharmaceuticals,1,None,NaN,NaN
2504,"entrinsic health solutions, inc.",1,None,NaN,NaN
2505,envisia therapeutics,1,None,NaN,NaN
2506,envoy medical corporation,1,None,NaN,NaN


In [167]:
pct_match = df_result.dropna()['count'].sum() / df_result['count'].sum()
pct_match

0.7176041395227637

### what is still remaining to be unmatched?

In [171]:
df_result[pd.isnull(df_result['matched_f'])][:20]

,name,count,matched_f,Ticker_f,source_f
7,boehringer ingelheim,584,None,NaN,NaN
44,ferring pharmaceuticals,87,None,NaN,NaN
52,bial - portela c s.a.,76,None,NaN,NaN
60,"coopervision, inc.",65,None,NaN,NaN
69,leo pharma,55,None,NaN,NaN
80,parexel,43,None,NaN,NaN
83,"mutual pharmaceutical company, inc.",42,None,NaN,NaN
84,"the emmes company, llc",41,None,NaN,NaN
86,grünenthal gmbh,41,None,NaN,NaN
87,purdue pharma lp,40,None,NaN,NaN


### all private companies ...